In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

[*********************100%%**********************]  1 of 1 completed


In [43]:

def get_data(ticker, start_date= '2000-01-01', end_date = '2024-22-03'):
    df = yf.download(ticker, start=start_date, end=end_date, interval='1wk')
    window_size = 3
    log_returns = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
    rolling_std_dev = log_returns.rolling(window=window_size).std()
    rolling_annualized_volatility = rolling_std_dev * np.sqrt(52)
    df['Volatility'] = rolling_annualized_volatility
    df['weekly_change'] = df['Adj Close'].pct_change() * 100
    return df

In [47]:
def get_volatility_range(df, num_std_devs = 1):
    curr_volatility = df['Volatility'][-1]
    std = df['Volatility'].std()
    return (curr_volatility - num_std_devs * std, curr_volatility + num_std_devs * std)

def filter_volatility_range(df, a, b):
    return df[(df['Volatility'] >= a) & (df['Volatility'] <= b)]


def get_bounds_for_running_prob(weekly_changes_within_volatility_range):
    bounds = {}
    for percentile in range(5, 50, 5):
        percentile = percentile / 100
        range_lower, range_upper = weekly_changes_within_volatility_range.quantile(percentile), weekly_changes_within_volatility_range.quantile(1-percentile)
        bounds[int((1-(percentile * 2)) * 100)] = (range_lower, range_upper)

    return [f"within {percent} percents, the weekly change is between {bound}" for percent, bound in bounds.items()]

def get_percent_for_bound(a,b, weekly_changes_within_volatility_range):
    return len(weekly_changes_within_volatility_range[(weekly_changes_within_volatility_range >= a) & (weekly_changes_within_volatility_range <= b)]) / len(weekly_changes_within_volatility_range) * 100



In [49]:
# Get the weekly changes within the recent volatility range
df = get_data('AAPL', '2020-01-01', '2021-01-01')
a, b = get_volatility_range(df)
weekly_changes_within_volatility_range = filter_volatility_range(df, a, b)['weekly_change']
print(get_bounds_for_running_prob(weekly_changes_within_volatility_range))

[*********************100%%**********************]  1 of 1 completed

['within 90 percents, the weekly change is between (-3.616833141176584, 7.624649856207166)', 'within 80 percents, the weekly change is between (-2.290431473186072, 6.786074271998066)', 'within 70 percents, the weekly change is between (-1.2354976308395216, 6.0416623302533115)', 'within 60 percents, the weekly change is between (-0.7744049216247606, 4.654513624536372)', 'within 50 percents, the weekly change is between (-0.1624508243355388, 4.120570887481262)', 'within 40 percents, the weekly change is between (0.1421906089995595, 3.1272931062630405)', 'within 30 percents, the weekly change is between (0.3566183347633034, 2.6984204877179394)', 'within 19 percents, the weekly change is between (0.8277438010400484, 2.267217255406462)', 'within 9 percents, the weekly change is between (1.301112438970159, 2.105912680934363)']



C:\Users\noama\AppData\Local\Temp\ipykernel_8324\3064302233.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  curr_volatility = df['Volatility'][-1]
